In [1]:
from Loader import fileloader as fl
from Loader import datahandledatabase as dhdb
import pandas as pd
import numpy as np
import time
import posixpath
from datetime import datetime
import openpyxl as xlsx

In [2]:
periodo = 201904

inidb = fl.ReadIniFile(mercado="empresas")
TESTPATH = inidb.getTestPath()

dbparser = inidb.getDbIniFileParser()

dbmanager = dhdb.DbDataProcess(str(periodo))
dbmanager.setParser(dbparser)

INFO:    Values loaded from globals.ini:
INFO:     * mainpath C:/Users/Johnny/OneDrive - Entel Peru S.A
INFO:     * scriptspath C:/Users/Johnny/Documents/Comisiones/Comisiones-python
INFO:     * databasepath C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB
INFO:    datapath value is C:/Users/Johnny/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is C:/Users/Johnny/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test
INFO:    Setting database path to C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


In [3]:
#To make sqlite dates recognized by external software we follow the ISO-8601 norm so
#date and time in the following format:  YYYY-MM-DD HH:MM:SS.SSS
def textToDateConverter(df, date_cols):
    for datevalue in date_cols:
        df[datevalue] = pd.to_datetime(df[datevalue],format='%Y-%m-%d', dayfirst = True)
    return df

In [4]:
# Equipos quincenales : actualizar
start_time = time.time()

# REPORTERIA ACTIVACIONES VOZ
#********************************************

HALF_MONTHTLY_TEAMS = []
DATE_COLUMNS = ['FECHA_PROCESO','FEC_ACTIV','FEC_DESACTIVA']
ORDER_COLUMN = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM','VENDEDOR',
                'FECHA_PROCESO','ESTADO']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']
CHANNEL_FILTER4 = ['SOLUCIONES DE DATOS']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA','COMISION_UNITARIA':'COMISION_UNITARIA_DB'}

COLS = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO','SUPERVISORKAM',
        'VENDEDOR','VENDEDOR_OTRO','POSICION_EMPL',
        'FECHA_PROCESO','CANALDEVENTA','CODIGO','RAZON_SOCIAL','CONTRATO','FEC_ACTIV','FEC_DESACTIVA', 'MODELO','TELEFONO',
        'ESTADO','TMCODE','PLAN_TARIFARIO','DIVISION','FECHA_ACTUALIZACION','TIPODOC','DOCUMENTO','GROSSCOMERCIAL',
        'GROSSCOMISION','ACCESS', 
        'ACCESSBOLSA','ACCESSPAQUETE','ACCESSLICENCIA', 'TIPOEQUIPO', 'RUC_10_20','NEWSADDS','TECNOLOGIAEQUIPO', 'NROORDEN', 
        'ACTIVIDAD','NUEVOSEGMENTO','CEDENTE', 'TIPOVENTA','DEPARTAMENTOGROSS','CARTERIZACION','FACTOR_FACTURACIÓN', 
        'DEALERS_REGIONES', 'COMISION_UNITARIA','PRIMA_SEGURO_ENTEL','ACCESSMASVEP','SEGMENTO_ACCESS','PORTA_EQUIPO']

## Revisar los contratos nulos en BD ##

#"""
ventas_ = dbmanager.loadData('View_Ventas')
ventas_ = textToDateConverter(ventas_, DATE_COLUMNS)
ventas_ = ventas_[COLS]

filter1 = ventas_['PERIODO_ACTIVACION'] == periodo
#filter2 = ventas_['PERIODO_ACTIVACION'] == end_date_dt.month + 201800 # Tener en cuenta diciembre
#filter3 = ventas_['ZONAVENTA'].isin(HALF_MONTHTLY_TEAMS)
#filter4 = ventas_['FECHA_PROCESO'] <= end_date_dt

ventas = ventas_[filter1]
#ventas_1 = ventas_[filter1]  #equipos de quincena
#ventast = ventas_[filter2 & filter3 & filter4] #equipos de quincena
#ventas = ventast_1.append(ventast, ignore_index = True) #equipos de quincena

stage1 = ventas['NEW_GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = ventas['GERENCIA1'].isin(CHANNEL_FILTER1)
stage3 = ventas['GERENCIA2'].isin(CHANNEL_FILTER3)
stage4 = ventas['GERENCIA2'].isin(CHANNEL_FILTER4)

#"""

#"""
# Filtrando las ventas_gce
#*************************

ventas_gce = ventas[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = ventas_gce['VENDEDOR_OTRO'] != ''
multiple_seller = ventas_gce[filter_reg]['VENDEDOR_OTRO'].values
ventas_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller
ventas_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_gce = ventas_gce.rename(columns = RENAME_COLUMNS)

ventas_gce.sort_values(by = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )

#"""

# Filtrando las ventas_gcp
#*************************

ventas_gcp = ventas[stage2 & ~ stage3]
order_col = ORDER_COLUMN.copy()
order_col.pop(0)
ventas_gcp.drop(DROP_COLUMNS2, axis =1, inplace = True)
ventas_gcp.sort_values(by = order_col,ascending = [1,1,1,1,1,1,1,1], inplace = True )

#"""

# Filtrando las ventas_datos
#*************************

ventas_datos = ventas[stage4]
ventas_datos.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_datos = ventas_datos.rename(columns = RENAME_COLUMNS)

ventas_datos.sort_values(by = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )

#"""

# REPORTERIA
ventas_.to_csv(posixpath.join(TESTPATH,str(periodo) + '_ventas_brutas_.csv'), index = False, encoding='utf-8-sig') # Punto de Control
ventas_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_gce.csv'), index = False, encoding='utf-8-sig')
ventas_gcp.to_csv(posixpath.join(TESTPATH ,str(periodo) + '_ventas_gcp.csv'), index = False, encoding='utf-8-sig')
ventas_datos.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_M2M.csv'), index = False, encoding='utf-8-sig')


time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

Los registros de la tabla View_Ventas es 32979 registros  


C:\Users\Johnny\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Johnny\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Johnny\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


--- 5.291694641113281 seconds ---


In [7]:
ventas

,PERIODO_ACTIVACION,GERENCIA1,GERENCIA2,NEW_GERENCIA2,ZONAVENTA,NEW_ZONAVENTA,DEPARTAMENTO,SUPERVISORKAM,VENDEDOR,VENDEDOR_OTRO,...,TIPOVENTA,DEPARTAMENTOGROSS,CARTERIZACION,FACTOR_FACTURACIÓN,DEALERS_REGIONES,COMISION_UNITARIA,PRIMA_SEGURO_ENTEL,ACCESSMASVEP,SEGMENTO_ACCESS,PORTA_EQUIPO
0,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,0,,,Venta_Chip
1,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
2,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
3,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
4,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
5,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
6,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
7,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
8,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip
9,201904,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,,REGIONAL NORTE,,ANCASH,CARTERA NORTE,MFLORES4,,...,Venta Normal,,,,,0.0,,,,Venta_Chip


In [4]:
# REPORTERIA ACTIVACIONES VAS NEW (SSAA)
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA2','ZONA_DE_VENTA',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER4 = ['DEALERS PYMES LIMA','DEALERS PYMES REGION','DESARROLLO NEGOCIOS PYME','VD PYMES']


COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA2','ZONA_DE_VENTA','VENDEDOR_CROSS_SELLING','ACTION_DATE',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESSREAL',
             'FAMILIA','GANADO_POR','EQUIPO','CUENTA','CONSIDERAR','OBSERVACIONES']

## Revisar los contratos nulos en BD ##

# Importing, cleaning, filtering and appending == > initial
ventas_ssaa_ = dbmanager.loadData('View_Ventas_SSAA_new')
ventas_ssaa_ = textToDateConverter(ventas_ssaa_, DATE_COLUMNS_DATA) #ok
ventas_ssaa_ = ventas_ssaa_[COLS_DATA]

filter1 = ventas_ssaa_['PERIODO_ACTIVACION'] == periodo
ventas_ssaa = ventas_ssaa_[filter1]

stage1 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER4)

#"""
# Filtrando las ventas_ssaa_gce
#*************************
ventas_ssaa_gce = ventas_ssaa[stage1]
ventas_ssaa_gce.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1], inplace = True )



#"""
# Filtrando las ventas_ssaa_gcp
#*************************

ventas_ssaa_gcp = ventas_ssaa[stage2]
ventas_ssaa_gcp.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1], inplace = True )

#"""
# REPORTERIA

ventas_ssaa.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_new.csv'), index = False, encoding='utf-8-sig') # Para Soluciones de Negocio
ventas_ssaa_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gce_new.csv'), index = False, encoding='utf-8-sig')
ventas_ssaa_gcp.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gcp_new.csv'), index = False, encoding='utf-8-sig')

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

Los registros de la tabla View_Ventas_SSAA_new es 9927 registros  
--- 0.6385722160339355 seconds ---


C:\Users\Johnny\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Johnny\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# REPORTERIA ACTIVACIONES VAS (SSAA)
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

#half_monthly_teams = ['MICHA','KRATOS','KAIZEN']
half_monthly_teams = []
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS_DATA ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA'}

COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO',
             'SUPERVISORKAM','VENDEDOR_CROSS_SELLING','VENDEDOR_OTRO','POSICION_EMPL','ACTION_DATE','CANALDEVENTA',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESS_REAL',
             'TIPO2','GANADO_POR','EQUIPO','CUENTA','OBSERVACIONES']

## Revisar los contratos nulos en BD ##

# Importing, cleaning, filtering and appending == > initial
ventas_ssaa_ = dbmanager.loadData('View_Ventas_SSAA')
ventas_ssaa_ = textToDateConverter(ventas_ssaa_, DATE_COLUMNS_DATA)
ventas_ssaa_ = ventas_ssaa_[COLS_DATA]


filter1 = ventas_ssaa_['PERIODO_ACTIVACION'] == periodo
filter2 = ventas_ssaa_['PERIODO_ACTIVACION'] == end_date_dt.month + 201800 # Tener en cuenta diciembre
filter3 = ventas_ssaa_['ZONAVENTA'].isin(half_monthly_teams)
filter4 = ventas_ssaa_['ACTION_DATE'] <= end_date_dt

ventas_ssaa_t_1 = ventas_ssaa_[filter1]
ventas_ssaa_t = ventas_ssaa_[filter2 & filter3 & filter4]
ventas_ssaa = ventas_ssaa_t_1.append(ventas_ssaa_t, ignore_index = True)

stage1 = ventas_ssaa['NEW_GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = ventas_ssaa['GERENCIA1'].isin(CHANNEL_FILTER1)
stage3 = ventas_ssaa['GERENCIA2'].isin(CHANNEL_FILTER3)

#"""
# Filtrando las ventas_ssaa_gce
#*************************
ventas_ssaa_gce = ventas_ssaa[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = ventas_ssaa_gce['VENDEDOR_OTRO'] != ''
multiple_seller = ventas_ssaa_gce[filter_reg]['VENDEDOR_OTRO'].values
ventas_ssaa_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller
ventas_ssaa_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
ventas_ssaa_gce = ventas_ssaa_gce.rename(columns = RENAME_COLUMNS_DATA)
ventas_ssaa_gce.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )


#"""

#"""
# Filtrando las ventas_ssaa_gcp
#*************************

ventas_ssaa_gcp = ventas_ssaa[stage2 & ~ stage3]
ventas_ssaa_gcp.drop(DROP_COLUMNS2, axis =1, inplace = True)
ventas_ssaa_gcp.sort_values(by = ORDER_COLUMN_DATA,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )

#"""

# REPORTERIA


ventas_ssaa_.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_.csv'), index = False, encoding='utf-8-sig') # Para Soluciones de Negocio
ventas_ssaa_gce.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gce.csv'), index = False, encoding='utf-8-sig')
ventas_ssaa_gcp.to_csv(posixpath.join(TESTPATH, str(periodo) + '_ventas_ssaa_gcp.csv'), index = False, encoding='utf-8-sig')

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)